# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
#import gmaps
import os

# Import API key
#from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Read weather_data.csv
weather_data = pd.read_csv(r'C:\Users\Mason\Data_Bootcamp\API-Weather\WeatherPy\weather_data.csv')

# Create DataFrame based on weather_data
weatherDF = pd.DataFrame(weather_data)
weatherDF

,City Name,Max Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,Beringovskiy,46.40,96,100,14.90,63.05,179.32
1,Nushki,66.65,56,0,1.74,29.55,66.02
2,Tiksi,77.00,92,100,24.49,71.69,128.87
3,Romitan,44.60,56,0,2.24,39.93,64.38
4,Upernavik,87.80,95,100,10.67,72.79,-56.15
...,...,...,...,...,...,...,...
65,Cabo San Lucas,44.60,58,20,9.17,22.89,-109.91
66,Barrow,87.80,92,90,12.75,71.29,-156.79
67,Thompson,24.80,62,90,9.17,55.74,-97.86
68,Ljubinje,23.00,93,75,2.24,42.95,18.09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
